In [5]:
%set_env ANTHROPIC_API_KEY=${ANTHROPIC_API_KEY}
%set_env LANGSMITH_TRACING=true
%set_env LANGSMITH_API_KEY=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
%set_env TAVILY_API_KEY=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


%set_env OPENAI_API_KEY=${OPENAI_API_KEY}
%set_env LANGCHAIN_API_KEY=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

%set_env LANGCHAIN_TRACING_V2=true
%set_env LANGCHAIN_PROJECT="Multi-agent Collaboration"

env: ANTHROPIC_API_KEY=${ANTHROPIC_API_KEY}
env: LANGSMITH_TRACING=true
env: LANGSMITH_API_KEY=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
env: TAVILY_API_KEY=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
env: OPENAI_API_KEY=${OPENAI_API_KEY}
env: LANGCHAIN_API_KEY=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
env: LANGCHAIN_TRACING_V2=true
env: LANGCHAIN_PROJECT="Multi-agent Collaboration"


In [2]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

# ${ANTHROPIC_API_KEY}

In [3]:
_set_env("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "Task Flow Manager"

# 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

In [2]:
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.schema import AIMessage, HumanMessage, SystemMessage

def ChatAnthropicSKT(model="anthropic/claude-3-5-sonnet-20240620"):
    llm_api_key = "sk-gapk-1tQCB4O2KnH5GG68DeUKfjAKQ-vJ9kc9"
    llm_api_url = "https://api.platform.a15t.com/v1"

    # model = "anthropic/claude-3-5-sonnet-20240620"

    model = ChatOpenAI(
        temperature=0,  
        openai_api_key=llm_api_key, 
        openai_api_base=llm_api_url, 
        model=model
        )
    return model

model = ChatAnthropicSKT()

model.invoke("Hello, how are you?")

INFO:httpx:HTTP Request: POST https://api.platform.a15t.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content="Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 13, 'total_tokens': 48}, 'model_name': 'claude-3-5-sonnet-20240620', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a356465f-ac07-47d2-9f98-55a5bee3b54c-0', usage_metadata={'input_tokens': 13, 'output_tokens': 35, 'total_tokens': 48})

In [99]:
from typing import List, Optional, Dict, Any, Annotated
import functools
import operator
import pandas as pd
import pandasql as psql
import json
import logging
from langchain.agents import AgentExecutor, Tool
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_anthropic import ChatAnthropic
from langgraph.graph import END, StateGraph
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from typing_extensions import TypedDict
from langchain.tools import StructuredTool
from langchain.chains import LLMChain
from langchain.agents import ZeroShotAgent
from langchain.output_parsers import RegexParser
from langchain.agents.agent import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish
import re
from typing import Union
from langgraph.graph import END, StateGraph, START

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Use Anthropic's ChatAnthropic
# llm = ChatAnthropic(model="claude-3-sonnet-20240229")
llm = ChatAnthropicSKT()

class SQLSearchTeamState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: List[str]
    next: Optional[str]
    dataframe_json: str
    df_schema: Optional[str]
    dataframe_path: str
    sql_query: Optional[str]
    search_result: Optional[str]

class FlexibleOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if the output is a conversational response
        if llm_output.strip().lower().startswith("i apologize") or llm_output.strip().lower().startswith("i'm sorry"):
            # Treat this as a request to check the schema
            return AgentAction(tool="check_schema", tool_input="", log=llm_output)
        
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # Parse out the action and action input
        action_match = re.search(r"Action:?\s*(.*?)[\n]*Action Input:?[\s]*(.*)", llm_output, re.DOTALL | re.IGNORECASE)
        if action_match:
            action = action_match.group(1).strip()
            action_input = action_match.group(2).strip(" ").strip('"')
            return AgentAction(tool=action, tool_input=action_input, log=llm_output)
        
        # If no action is found, but there are thoughts about checking the schema, default to check_schema
        thoughts_match = re.search(r"Thoughts:?\s*(.*)", llm_output, re.DOTALL | re.IGNORECASE)
        if thoughts_match and ("schema" in thoughts_match.group(1).lower() or "structure" in thoughts_match.group(1).lower()):
            return AgentAction(tool="check_schema", tool_input="", log=llm_output)
        
        # If nothing matches, raise an error
        raise ValueError(f"Could not parse LLM output: `{llm_output}`")
    
def create_agent(llm: ChatAnthropic, tools: List[StructuredTool], system_prompt: str) -> AgentExecutor:
    tool_names = [tool.name for tool in tools]
    tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
    
    template = f"""
    {system_prompt}
    
    You have access to the following tools:
    {tool_descriptions}
    
    Use the following format:

    Thoughts: your thoughts about the task
    Action: the action to take, must be one of {tool_names}
    Action Input: the input to the action
    
    If you have completed the task, use this format:
    
    Thoughts: your final thoughts about the task
    Final Answer: the final answer to the task
    
    Remember, you must always use one of the above formats. Do not respond conversationally.
    If you're unsure or need more information, use the 'check_schema' action.
    
    Human: {{input}}
    AI: Let's approach this step-by-step:
    """
    
    prompt = PromptTemplate(
        template=template,
        input_variables=["input"],
    )
    
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    tools = [Tool(name=tool.name, func=tool.func, description=tool.description) for tool in tools]
    
    output_parser = FlexibleOutputParser()
    
    agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names, output_parser=output_parser)
    
    return AgentExecutor.from_agent_and_tools(
        agent=agent, 
        tools=tools, 
        verbose=True, 
        handle_parsing_errors=True,
        max_iterations=5  # Limit the number of iterations
    )

def agent_node(state: SQLSearchTeamState, agent: AgentExecutor, name: str) -> Dict[str, Any]:
    logger.info(f"{name} agent called with state: {state}")
    
    if name == "Schema Analyst":
        agent_input = f"""
        Current state:
        DataFrame JSON: {state['dataframe_json'][:1000] if isinstance(state['dataframe_json'], str) else str(state['dataframe_json'])[:1000]}...  # Truncate for readability
        DataFrame path: '{state['dataframe_path']}'
        Your task as Schema Analyst: Analyze the schema of the given DataFrame JSON or loaded from file path.
        """
    elif name == "SQL Expert":
        agent_input = f"""
        Current state:
        DataFrame Schema: {state['df_schema']}
        User Request: {state['messages'][0].content}
        
        Your task as SQL Expert: Generate a SQL query to {state['messages'][0].content} based on the given schema.
        """
    else:
        agent_input = f"""
        Current state:
        DataFrame JSON: {state['dataframe_json'][:1000] if isinstance(state['dataframe_json'], str) else str(state['dataframe_json'])[:1000]}...  # Truncate for readability
        DataFrame Schema: {state['df_schema']}
        SQL Query: {state['sql_query']}
        
        Your task as {name}: {state['messages'][0].content}
        """

    logger.info(f">>> {name} agent input: {agent_input}")

    try:
        result = agent.invoke({"input": agent_input})
        output = result["output"]
        print(f">>>>>>> {name} agent output: {output}")
    except Exception as e:
        logger.error(f"Error in agent execution: {e}")
        output = f"Error: {str(e)}. Defaulting to previous state."
        print(f"====== {name} agent output: {output}")
        
    new_state = state.copy()
    new_state["messages"] = state["messages"] + [HumanMessage(content=output, name=name)]
    
    if name == "Schema Analyst":
        new_state["df_schema"] = output if "DataFrame schema:" in output else state['df_schema']
    elif name == "SQL Expert":
        new_state["sql_query"] = output
    
    return new_state

def team_supervisor(state: SQLSearchTeamState) -> Dict[str, Any]:
    logger.info(f"Team supervisor called with state: {state}")
    options = ["FINISH"] + state["team_members"]
    
    # Check if we have a schema and no SQL query yet
    if state['df_schema'] and not state['sql_query']:
        logger.info("Schema is available and SQL query is not generated yet. Selecting SQL Expert.")
        return {"next": "SQL Expert"}
    
    # If we have both schema and SQL query, we can finish
    if state['df_schema'] and state['sql_query']:
        logger.info("Both schema and SQL query are available. Finishing the process.")
        return {"next": "FINISH"}
    
    prompt = f"""
    You are the supervisor of a team of data analysts. Your job is to coordinate the team's efforts to analyze a pandas DataFrame based on user input. The team consists of a Schema Analyst and SQL Expert.

    Current state:
    DataFrame Schema: {state['df_schema']}
    SQL Query: {state['sql_query']}
    
    Given the current state, who should act next? Or should we FINISH?
    Select ONLY ONE of the following options: {', '.join(options)}.
    Respond with just the name, nothing else.
    """
        
    ai_message = llm.invoke([HumanMessage(content=prompt)]).content.strip()
        
    if ai_message not in options:
        logger.warning(f"Invalid response from AI: {ai_message}. Defaulting to FINISH.")
        ai_message = "FINISH"
    
    logger.info(f"Team supervisor decided next action: {ai_message}")
    return {"next": ai_message}

# Define tools for each step
def check_schema(dataframe_json, dataframe_path) -> str:    
    try:
        print("----------------- dataframe_json:", dataframe_json)
        print("----------------- dataframe_path:", dataframe_path)
        # If it's a string, try to parse it as JSON
        try:
            if isinstance(dataframe_json, str):
                json_data = json.loads(dataframe_json)
            else:
                json_data = dataframe_json
            
            # Create a DataFrame
            df = pd.DataFrame(json_data)
        except:
            df = pd.read_csv(dataframe_path)
        
        # Get the schema
        schema = df.dtypes.to_string()
        
        logger.info(f"Schema successfully extracted: {schema}")
        return f"DataFrame schema:\n{schema}"
    except json.JSONDecodeError as e:
        error_msg = f"Error decoding JSON string: {str(e)}"
        logger.error(error_msg)
        return error_msg
    except Exception as e:
        error_msg = f"Error checking schema: {str(e)}"
        logger.error(error_msg)
        return error_msg
      
def generate_sql_query(df_schema: str, user_input: str) -> str:
    print("============== df_schema:", df_schema)
    try:
        prompt = f"Given the following schema:\n{df_schema}\n\nGenerate a SQL query for the following request:\n{user_input}"
        sql_query = llm.invoke([HumanMessage(content=prompt)]).content
        return sql_query
    except Exception as e:
        logger.error(f"Error generating SQL query: {e}")
        return ""

def search_dataframe(dataframe_json: str, sql_query: str) -> str:
    try:
        df = pd.read_json(dataframe_json)
        result = psql.sqldf(sql_query, locals())
        return result.to_json(orient='records')
    except Exception as e:
        logger.error(f"Error searching dataframe: {e}")
        return "[]"

# Create StructuredTools
check_schema_tool = StructuredTool.from_function(
    func=check_schema,
    name="check_schema",
    description="Check the schema of a pandas DataFrame"
)

generate_sql_tool = StructuredTool.from_function(
    func=generate_sql_query,
    name="generate_sql_query",
    description="Generate a SQL query based on the schema and user input"
)

search_dataframe_tool = StructuredTool.from_function(
    func=search_dataframe,
    name="search_dataframe",
    description="Search a pandas DataFrame using a SQL query"
)

# Create agents for each step
schema_agent = create_agent(llm, [check_schema_tool], "You are a data analyst specializing in understanding data schemas.")
sql_agent = create_agent(llm, [generate_sql_tool], "You are a SQL expert who generates queries based on natural language requests and data schemas.")
search_agent = create_agent(llm, [search_dataframe_tool], "You are a data scientist who executes SQL queries on pandas DataFrames and interprets the results.")

# Create the workflow
workflow = StateGraph(SQLSearchTeamState)

# Add nodes
workflow.add_node("check_schema", lambda state: agent_node(state, schema_agent, "Schema Analyst"))
workflow.add_node("generate_sql", lambda state: agent_node(state, sql_agent, "SQL Expert"))
# workflow.add_node("search_dataframe", lambda state: agent_node(state, search_agent, "Data Scientist"))
workflow.add_node("team_supervisor", team_supervisor)

# Add edges
# workflow.add_edge(START, "team_supervisor")

workflow.add_edge("check_schema", "team_supervisor")
workflow.add_edge("generate_sql", "team_supervisor")
# workflow.add_edge("search_dataframe", "team_supervisor")

# Set conditional edges
workflow.set_entry_point("check_schema")
workflow.add_conditional_edges(
    "team_supervisor",
    lambda x: x["next"],
    {
        "Schema Analyst": "check_schema",
        "SQL Expert": "generate_sql",
        # "Data Scientist": "search_dataframe",
        "FINISH": END
    }
)

# Compile the graph
app = workflow.compile()

# Example usage
df = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'City': ['New York', 'San Francisco', 'Los Angeles']
})

initial_state = SQLSearchTeamState(
    messages=[HumanMessage(content="Find all people over 30 years old")],
    team_members=[
        "Schema Analyst",
        "SQL Expert", 
        # "Data Scientist"
                  ],
    next=None,
    dataframe_json=df.to_json(orient='records'),
    dataframe_path="data/example.csv",
    df_schema=None,
    sql_query=None,
    search_result=None
)

logging.basicConfig(level=logging.INFO)

i = 0
for output in app.stream(initial_state):
    print("Workflow output:", output)
    if i > 20:
        break
    i += 1

INFO:__main__:Schema Analyst agent called with state: {'messages': [HumanMessage(content='Find all people over 30 years old')], 'team_members': ['Schema Analyst', 'SQL Expert'], 'next': None, 'dataframe_json': '[{"Name":"Alice","Age":25,"City":"New York"},{"Name":"Bob","Age":30,"City":"San Francisco"},{"Name":"Charlie","Age":35,"City":"Los Angeles"}]', 'df_schema': None, 'dataframe_path': 'data/example.csv', 'sql_query': None, 'search_result': None}
INFO:__main__:>>> Schema Analyst agent input: 
        Current state:
        DataFrame JSON: [{"Name":"Alice","Age":25,"City":"New York"},{"Name":"Bob","Age":30,"City":"San Francisco"},{"Name":"Charlie","Age":35,"City":"Los Angeles"}]...  # Truncate for readability
        DataFrame path: 'data/example.csv'
        Your task as Schema Analyst: Analyze the schema of the given DataFrame JSON.
        




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.platform.a15t.com/v1/chat/completions "HTTP/1.1 200 OK"
ERROR:__main__:Error in agent execution: check_schema() missing 1 required positional argument: 'dataframe_path'
INFO:__main__:Team supervisor called with state: {'messages': [HumanMessage(content='Find all people over 30 years old'), HumanMessage(content='Find all people over 30 years old'), HumanMessage(content="Error: check_schema() missing 1 required positional argument: 'dataframe_path'. Defaulting to previous state.", name='Schema Analyst')], 'team_members': ['Schema Analyst', 'SQL Expert'], 'next': None, 'dataframe_json': '[{"Name":"Alice","Age":25,"City":"New York"},{"Name":"Bob","Age":30,"City":"San Francisco"},{"Name":"Charlie","Age":35,"City":"Los Angeles"}]', 'df_schema': None, 'dataframe_path': 'data/example.csv', 'sql_query': None, 'search_result': None}


Thoughts: To analyze the schema of the given DataFrame JSON, I need to use the check_schema tool. This will provide me with detailed information about the structure and data types of the DataFrame.

Action: check_schema
Action Input:====== Schema Analyst agent output: Error: check_schema() missing 1 required positional argument: 'dataframe_path'. Defaulting to previous state.
Workflow output: {'check_schema': {'messages': [HumanMessage(content='Find all people over 30 years old'), HumanMessage(content="Error: check_schema() missing 1 required positional argument: 'dataframe_path'. Defaulting to previous state.", name='Schema Analyst')], 'team_members': ['Schema Analyst', 'SQL Expert'], 'next': None, 'dataframe_json': '[{"Name":"Alice","Age":25,"City":"New York"},{"Name":"Bob","Age":30,"City":"San Francisco"},{"Name":"Charlie","Age":35,"City":"Los Angeles"}]', 'df_schema': None, 'dataframe_path': 'data/example.csv', 'sql_query': None, 'search_result': None}}


INFO:httpx:HTTP Request: POST https://api.platform.a15t.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Team supervisor decided next action: Schema Analyst
INFO:__main__:Schema Analyst agent called with state: {'messages': [HumanMessage(content='Find all people over 30 years old'), HumanMessage(content='Find all people over 30 years old'), HumanMessage(content="Error: check_schema() missing 1 required positional argument: 'dataframe_path'. Defaulting to previous state.", name='Schema Analyst')], 'team_members': ['Schema Analyst', 'SQL Expert'], 'next': 'Schema Analyst', 'dataframe_json': '[{"Name":"Alice","Age":25,"City":"New York"},{"Name":"Bob","Age":30,"City":"San Francisco"},{"Name":"Charlie","Age":35,"City":"Los Angeles"}]', 'df_schema': None, 'dataframe_path': 'data/example.csv', 'sql_query': None, 'search_result': None}
INFO:__main__:>>> Schema Analyst agent input: 
        Current state:
        DataFrame JSON: [{"Name":"Alice","Age":25,"City":"New York"},{"Name":"Bob",

...

Workflow output: {'team_supervisor': {'next': 'Schema Analyst'}}


> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.platform.a15t.com/v1/chat/completions "HTTP/1.1 200 OK"
ERROR:__main__:Error in agent execution: check_schema() missing 1 required positional argument: 'dataframe_path'
INFO:__main__:Team supervisor called with state: {'messages': [HumanMessage(content='Find all people over 30 years old'), HumanMessage(content='Find all people over 30 years old'), HumanMessage(content="Error: check_schema() missing 1 required positional argument: 'dataframe_path'. Defaulting to previous state.", name='Schema Analyst'), HumanMessage(content='Find all people over 30 years old'), HumanMessage(content='Find all people over 30 years old'), HumanMessage(content="Error: check_schema() missing 1 required positional argument: 'dataframe_path'. Defaulting to previous state.", name='Schema Analyst'), HumanMessage(content="Error: check_schema() missing 1 required positional argument: 'dataframe_path'. Defaulting to previous state.", name='Schema Analyst'), HumanMessage(con

Thoughts: To analyze the schema of the given DataFrame JSON, I need to use the check_schema tool. This will provide me with detailed information about the structure and data types of the DataFrame.

Action: check_schema
Action Input:====== Schema Analyst agent output: Error: check_schema() missing 1 required positional argument: 'dataframe_path'. Defaulting to previous state.
Workflow output: {'check_schema': {'messages': [HumanMessage(content='Find all people over 30 years old'), HumanMessage(content='Find all people over 30 years old'), HumanMessage(content="Error: check_schema() missing 1 required positional argument: 'dataframe_path'. Defaulting to previous state.", name='Schema Analyst'), HumanMessage(content='Find all people over 30 years old'), HumanMessage(content='Find all people over 30 years old'), HumanMessage(content="Error: check_schema() missing 1 required positional argument: 'dataframe_path'. Defaulting to previous state.", name='Schema Analyst'), HumanMessage(content=

INFO:httpx:HTTP Request: POST https://api.platform.a15t.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Team supervisor decided next action: Schema Analyst


Workflow output: {'team_supervisor': {'next': 'Schema Analyst'}}
